In [1]:
import pickle 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import scipy.io
from cv2 import  copyMakeBorder, BORDER_CONSTANT

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, BatchNormalization, Dropout

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing joints locations and images - correcting size of the images

In [2]:
coords_file_name = "/content/drive/My Drive/Deep/HW02/lsp_dataset/joints.mat"
joints = scipy.io.loadmat(coords_file_name)
joints = joints['joints'][:2,:]

image_list = []

for i in range(1,2001):
  if np.mod(i,50)==0:
    print(i)
  if i<10:
    img_name = "im000{0}.jpg".format(i)
  elif i<100:
    img_name = "im00{0}.jpg".format(i)
  elif i<1000:
    img_name = "im0{0}.jpg".format(i)
  else:
    img_name = "im{0}.jpg".format(i)

  img = plt.imread("/content/drive/My Drive/Deep/HW02/lsp_dataset/images/{0}".format(img_name))

  if img.shape[0] < 220:
    height = 220 - img.shape[0]
  else:
    height = 0

  if img.shape[1] < 220:
    width = 220 - img.shape[1]
  else:
    width = 0
  
  if np.mod(width,2) == 0:
    extra_width = 0
  else:
    extra_width = 1

  if np.mod(height,2) == 0:
    extra_height = 0
  else:
    extra_height = 1

  joints[0,:,i-1] = (joints[0,:,i-1] + width/2)/220
  joints[1,:,i-1] = (joints[1,:,i-1] + height/2)/220

  img = copyMakeBorder(img,int(height/2)+extra_height,int(height/2),int(width/2)+extra_width,int(width/2),BORDER_CONSTANT,value=[0,0,0])

  image_list.append(img/255)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000


### Separating train, test, and validation

In [3]:
X_train = image_list[0:1400]
Y_train = joints[:,:,0:1400]

X_train = np.reshape(X_train,(1400,220,220,3))
Y_train = np.reshape(Y_train,(1400,28))

X_validation = image_list[1400:1700]
Y_validation = joints[:,:,1400:1700]

X_validation = np.reshape(X_validation,(300,220,220,3))
Y_validation = np.reshape(Y_validation,(300,28))

X_test = image_list[1700:]
Y_test = joints[:,:,1700:]

X_test = np.reshape(X_validation,(300,220,220,3))
Y_test = np.reshape(Y_test,(300,28))

### Running The DNN

In [10]:
batch_size = 64
epochs_number = 40
learning_rate = 1e-3
momentum = 0.9


model = tf.keras.models.Sequential()

model.add(Conv2D(48, (11, 11), activation='relu', strides=4, kernel_initializer='he_uniform', padding='same',  input_shape=(220, 220, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (5, 5), activation='relu', strides=2, kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(192, (3, 3), activation='relu', strides=2, kernel_initializer='he_uniform', padding='same'))

model.add(Conv2D(192, (3, 3), activation='relu', strides=1, kernel_initializer='he_uniform', padding='same'))

model.add(Conv2D(192, (3, 3), activation='relu', strides=1, kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))

model.add(Dense(4096, activation='relu'))

model.add(Dense(28, activation='linear'))


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=epochs_number, batch_size=batch_size, validation_data=(X_validation, Y_validation))

test_results = model.evaluate(X_test, Y_test, verbose=1)
print('\n', "Test results - Loss: {0:.2f} - Accuracy: {1:.2f}%".format(test_results[0],test_results[1]))

Epoch 1/40
22/22 [==============================] - 4s 123ms/step - loss: 9.7482 - accuracy: 0.0400 - val_loss: 0.1183 - val_accuracy: 0.0367
Epoch 2/40
22/22 [==============================] - 2s 76ms/step - loss: 0.0233 - accuracy: 0.0471 - val_loss: 0.0351 - val_accuracy: 0.0300
Epoch 3/40
22/22 [==============================] - 2s 81ms/step - loss: 0.0221 - accuracy: 0.0421 - val_loss: 0.0492 - val_accuracy: 0.0300
Epoch 4/40
22/22 [==============================] - 2s 81ms/step - loss: 0.0206 - accuracy: 0.0379 - val_loss: 0.0345 - val_accuracy: 0.0367
Epoch 5/40
22/22 [==============================] - 2s 81ms/step - loss: 0.0212 - accuracy: 0.0400 - val_loss: 0.0376 - val_accuracy: 0.0400
Epoch 6/40
22/22 [==============================] - 2s 81ms/step - loss: 0.0202 - accuracy: 0.0443 - val_loss: 0.0353 - val_accuracy: 0.0400
Epoch 7/40
22/22 [==============================] - 2s 77ms/step - loss: 0.0203 - accuracy: 0.0436 - val_loss: 0.0361 - val_accuracy: 0.0300
Epoch 8/40
 

KeyboardInterrupt: ignored